In [9]:
import pandas as pd

df = pd.read_csv("ad_clicks.csv")
df.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


### Grupping user ID data by utm_source.

In [10]:
utm_counts = df.groupby("utm_source").user_id.count().reset_index()
utm_counts

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


### Creating new column determinig whether clicks are done or not.

In [11]:
df['is_click'] = ~df.ad_click_timestamp.isnull()
df.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


### Gaining the percent of people who clicked on ads from each source

In [12]:
click_by_source = df.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
click_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


In [13]:
click_pivot = click_by_source.pivot(
    columns = 'is_click', 
    index = 'utm_source', 
    values = 'user_id').reset_index()
click_pivot

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


In [14]:
click_pivot['percent_click'] = click_pivot[True] / (click_pivot[True] + click_pivot[False])
click_pivot.percent_click

0    0.313725
1    0.357143
2    0.351471
3    0.306977
Name: percent_click, dtype: float64

### Analysing A/B test

In [15]:
exp_group = df.groupby('experimental_group').count().reset_index()
exp_group

,experimental_group,user_id,utm_source,day,ad_click_timestamp,is_click
0,A,827,827,827,310,827
1,B,827,827,827,255,827


In [16]:
print(df\
      .groupby(['experimental_group', 'is_click'])\
      .user_id.count().reset_index()\
      .pivot(
        columns = 'is_click',
        index = 'experimental_group',
        values = 'user_id')\
      .reset_index())

is_click experimental_group  False  True
0                         A    517   310
1                         B    572   255


### Creating separated dataframes containing results from group A and group B.

In [31]:
a_results = df[df.experimental_group == 'A']
b_results = df[df.experimental_group == 'B']

a_results.head()
b_results.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False


### Gaining the percent of users who clicked on the ad by day in each group.

### Group A

In [17]:
a_results_piv = df\
    .groupby(['is_click', 'day'])\
    .user_id.count()\
    .reset_index()\
    .pivot(
        columns = 'is_click',
        index = 'day',
        values = 'user_id')\
    .reset_index()
a_results_piv

is_click,day,False,True
0,1 - Monday,151,75
1,2 - Tuesday,150,88
2,3 - Wednesday,175,73
3,4 - Thursday,156,76
4,5 - Friday,167,89
5,6 - Saturday,149,87
6,7 - Sunday,141,77


In [20]:
a_results_piv["percent_clicked"] = a_results_piv[False] / (a_results_piv[False] + a_results_piv[True])
a_results_piv

is_click,day,False,True,percent_clicked
0,1 - Monday,151,75,0.668142
1,2 - Tuesday,150,88,0.630252
2,3 - Wednesday,175,73,0.705645
3,4 - Thursday,156,76,0.672414
4,5 - Friday,167,89,0.652344
5,6 - Saturday,149,87,0.631356
6,7 - Sunday,141,77,0.646789


### Group B

In [22]:
b_results_piv = df\
    .groupby(['is_click', 'day'])\
    .user_id.count()\
    .reset_index()\
    .pivot(
        columns = 'is_click',
        index = 'day',
        values = 'user_id')\
    .reset_index()
b_results_piv

is_click,day,False,True
0,1 - Monday,151,75
1,2 - Tuesday,150,88
2,3 - Wednesday,175,73
3,4 - Thursday,156,76
4,5 - Friday,167,89
5,6 - Saturday,149,87
6,7 - Sunday,141,77


In [23]:
b_results_piv['percent_clicked'] = b_results_piv[True] / (b_results_piv[True] + b_results_piv[False])
b_results_piv

is_click,day,False,True,percent_clicked
0,1 - Monday,151,75,0.331858
1,2 - Tuesday,150,88,0.369748
2,3 - Wednesday,175,73,0.294355
3,4 - Thursday,156,76,0.327586
4,5 - Friday,167,89,0.347656
5,6 - Saturday,149,87,0.368644
6,7 - Sunday,141,77,0.353211


### The result is that version A is preferred over version B.